In [34]:
import requests
from bs4 import BeautifulSoup

In [35]:
doc_url = "https://docs.google.com/document/d/1Ga6VNnVF3328Q-ytXMbwgup42Jzh6DEPTbmf2RHVH6A/edit?tab=t.0"

In [32]:
def get_doc(doc_url):
    base_url = doc_url.split('/edit')[0] if '/edit' in doc_url else doc_url.split('?')[0]
    exp_url = base_url + '/export?format=html'
    
    response = requests.get(exp_url)
    
    response.raise_for_status()
    return response.text

In [30]:
def parse_doc(doc_text):
    grid = {}
    max_x = 0
    max_y = 0
    
    for line in doc_text.strip().splitlines():
        parts = line.strip().split()
        if len(parts) < 3:
            continue
        x_str, char, y_str = parts[0], parts[1], parts[2]
        try:
            x = int(x_str)
            y = int(y_str)
        except ValueError:
            continue
        
        grid[(x, y)] = char
        max_x = max(max_x, x)
        max_y = max(max_y, y)
    
    # Print the grid with y increasing downward, x increasing rightward
    for y in range(max_y + 1):
        row_chars = []
        for x in range(max_x + 1):
            row_chars.append(grid.get((x, y), ' '))
        print(''.join(row_chars))

In [14]:
def print_grid(doc_url):
    doc_text = get_doc(doc_url)
    parse_doc(doc_text)

In [33]:
test = print_grid(doc_url)
test

In [47]:
# Required Dependencies.
import requests
from bs4 import BeautifulSoup

# Defining a function to take a google doc and parse the table to decode a secret message.
def print_grid_from_table(doc_url):
    """
    Downloads and parses a published Google Doc table containing character coordinates, then prints a secret message.

    Args:
        doc_url (str): URL of the Google Doc containing a table with columns: x-coordinate, character, y-coordinate.

    Prints:
        The grid of characters arranged by their (x, y) positions, filling empty spots with spaces.
    """
    
    # Fetch the doc as html.
    response = requests.get(doc_url)
    response.raise_for_status()
    html = response.text
    
    # Using BeautifulSoup html parser to examine table in the response.
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    # Error checking.
    if not table:
        print("No table found in document!")
        return
    
    # Find table rows.
    rows = table.find_all('tr')
    
    # Create a dict to store characters in a grid.
    grid = {}
    max_x = 0
    max_y = 0
    
    # Skip header row and parse each row for x, char, y.
    for row in rows[1:]:
        cells = row.find_all(['td', 'th'])
        # Ensuring rows have three cols of data.
        if len(cells) < 3:
            continue
        x_str = cells[0].get_text(strip=True)
        char = cells[1].get_text(strip=True)
        y_str = cells[2].get_text(strip=True)
        
        # Data validation for rows with incorrectly formatted data in x / y coords.
        try:
            x = int(x_str)
            y = int(y_str)
        except ValueError:
            continue
        
        # Storing the character at the correct position.
        grid[(x, y)] = char
        max_x = max(max_x, x)
        max_y = max(max_y, y)
    
    # Print rows from top (y=0) to bottom (max_y)
    for y in range(max_y, -1, -1):
        row_str = ''.join(grid.get((x, y), ' ') for x in range(max_x + 1))
        print(row_str)

In [44]:
test2 = print_grid_from_table(doc_url)

█▀▀▀
█▀▀ 
█   


In [48]:
print_grid_from_table("https://docs.google.com/document/d/e/2PACX-1vQGUck9HIFCyezsrBSnmENk5ieJuYwpt7YHYEzeNJkIb9OSDdx-ov2nRNReKQyey-cwJOoEKUhLmN9z/pub")


████████░     ████████░   ██████████░    ███████░  ██░           ███░ ███░    ███░ ██░     ██░
██░     ██░ ███░     ███░ ██░          ███░    ██░ ███░   ███░   ██░    ██░  ██░   ██░     ██░
██░     ██░ ██░       ██░ ██░         ███░          ██░  █████░ ███░     ██░██░    ██░     ██░
████████░   ██░       ██░ ████████░   ██░           ███░ ██░██░ ██░       ███░     ██████████░
██░     ██░ ██░       ██░ ██░         ███░           ██░██░ ██░██░       ██░██░    ██░     ██░
██░     ██░ ███░     ███░ ██░          ███░    ██░   ████░   ████░      ██░  ██░   ██░     ██░
████████░     ████████░   ██████████░    ███████░     ██░     ██░     ███░    ███░ ██░     ██░
